In [9]:
import os
import cv2
import numpy as np
import random

def apply_mosaic(image, location=(50, 50), mosaic_size=(50, 50), shape='circle'):
    """
    Applies a mosaic effect to an image using different shapes.
    
    Parameters:
    - image (np.array): Input image.
    - location (tuple): (x, y) coordinates of the top-left corner where the mosaic will be applied.
    - mosaic_size (tuple): (width, height) of the mosaic area.
    - shape (str): Type of shape ('circle', 'rectangle', 'triangle').
    
    Returns:
    - Processed image with mosaic effect.
    """
    x, y = location
    mosaic_width, mosaic_height = mosaic_size
    height, width = image.shape[:2]

    # Ensure the coordinates are within bounds
    if x + mosaic_width > width or y + mosaic_height > height:
        raise ValueError("Mosaic area exceeds image boundaries.")

    # Extract the region of interest (ROI)
    roi = image[y:y + mosaic_height, x:x + mosaic_width]

    if roi.size != 0:  # Check if the region is valid
        # Pixelate the area within the region of interest (ROI)
        small = cv2.resize(roi, (10, 10), interpolation=cv2.INTER_LINEAR)
        pixelated = cv2.resize(small, (mosaic_width, mosaic_height), interpolation=cv2.INTER_NEAREST)

        # Create a mask to determine where to apply the mosaic based on the shape
        mask = np.zeros((mosaic_height, mosaic_width), dtype=np.uint8)

        if shape == 'circle':
            # Draw a filled circle on the mask
            center = (mosaic_width // 2, mosaic_height // 2)
            radius = min(mosaic_width, mosaic_height) // 2
            cv2.circle(mask, center, radius, 255, -1)

        elif shape == 'rectangle':
            # Draw a filled rectangle on the mask
            cv2.rectangle(mask, (0, 0), (mosaic_width, mosaic_height), 255, -1)

        elif shape == 'triangle':
            # Draw a filled triangle on the mask
            pts = np.array([
                [0, mosaic_height],
                [mosaic_width // 2, 0],
                [mosaic_width, mosaic_height]
            ], np.int32)
            cv2.drawContours(mask, [pts], 0, 255, -1)

        # Blend the mosaic region with the original ROI using the mask
        blended_roi = np.where(mask[:, :, None] == 255, pixelated, roi)

        # Replace the original image's ROI with the blended ROI
        image[y:y + mosaic_height, x:x + mosaic_width] = blended_roi

    return image

def resize_image(image, target_size=(400, 400)):
    """
    Resizes the image to a fixed target size.
    
    Parameters:
    - image (np.array): Input image.
    - target_size (tuple): (width, height) to resize the image.
    
    Returns:
    - Resized image.
    """
    return cv2.resize(image, target_size, interpolation=cv2.INTER_AREA)

def process_dataset(input_dir, output_dir, target_size=(400, 400), mosaic_location=(100, 100), mosaic_size=(80, 80)):
    """
    Processes a dataset of images by resizing and applying a mosaic effect with different shapes.
    
    Parameters:
    - input_dir (str): Directory containing input images.
    - output_dir (str): Directory to save processed images.
    - target_size (tuple): (width, height) to resize the images.
    - mosaic_location (tuple): (x, y) coordinates for the mosaic shape.
    - mosaic_size (tuple): (width, height) of the mosaic shape.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    shapes = ['circle', 'rectangle', 'triangle']

    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)
            
            # Read the image using OpenCV
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error loading image {filename}. Skipping...")
                continue

            # Resize the image to the target size
            resized_image = resize_image(image, target_size)

            # Choose a random shape for the mosaic
            shape = random.choice(shapes)

            # Apply the mosaic effect
            result = apply_mosaic(resized_image, mosaic_location, mosaic_size, shape)

            # Save the processed image
            cv2.imwrite(output_path, result)
            print(f"Processed and saved: {output_path}")

# Example usage
input_directory = '../data/dataset/Yorkshire_Terrier'
output_directory = '../data/label/Yorkshire_Terrier'
process_dataset(
    input_directory,
    output_directory,
    target_size=(400, 400),  # Resize images to 400x400 for better visibility
    mosaic_location=(150, 150),  # Adjust mosaic position for a centered effect
    mosaic_size=(100, 100)  # Mosaic size to match the updated image size
)

Processed and saved: ../data/label/Yorkshire_Terrier/Yorkshire Terrier_17.jpg
Processed and saved: ../data/label/Yorkshire_Terrier/Yorkshire Terrier_16.jpg
Processed and saved: ../data/label/Yorkshire_Terrier/Yorkshire Terrier_28.jpg
Processed and saved: ../data/label/Yorkshire_Terrier/Yorkshire Terrier_14.jpg
Processed and saved: ../data/label/Yorkshire_Terrier/Yorkshire Terrier_15.jpg
Processed and saved: ../data/label/Yorkshire_Terrier/Yorkshire Terrier_29.jpg
Processed and saved: ../data/label/Yorkshire_Terrier/Yorkshire Terrier_11.jpg
Processed and saved: ../data/label/Yorkshire_Terrier/Yorkshire Terrier_39.jpg
Processed and saved: ../data/label/Yorkshire_Terrier/Yorkshire Terrier_38.jpg
Processed and saved: ../data/label/Yorkshire_Terrier/Yorkshire Terrier_10.jpg
Processed and saved: ../data/label/Yorkshire_Terrier/Yorkshire Terrier_12.jpg
Processed and saved: ../data/label/Yorkshire_Terrier/Yorkshire Terrier_13.jpg
Processed and saved: ../data/label/Yorkshire_Terrier/Yorkshire T